<!-- This capstone project will be graded by your peers. This capstone project is worth 70% of your total grade. The project will be completed over the course of 2 weeks.  Week 1 submissions will be worth 30% whereas week 2 submissions will be worth 40% of your total grade.

For this week, you will required to submit the following:

A description of the problem and a discussion of the background. (15 marks)
A description of the data and how it will be used to solve the problem. (15 marks)
For the second week, the final deliverables of the project will be:

A link to your Notebook on your Github repository, showing your code. (15 marks)
2. A full report consisting of all of the following components (15 marks):

Introduction where you discuss the business problem and who would be interested in this project.
Data where you describe the data that will be used to solve the problem and the source of the data.
Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.
Results section where you discuss the results.
Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.
Conclusion section where you conclude the report.
3. Your choice of a presentation or blogpost. (10 marks) -->

# Name of the Project:The Battle of Neighborhoods (Week 2)

## Data extraction, cleaning, analysis, and reporting


### Data source.

A. For relevant data of Manhattan, we use the following link
"https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json"

which gives us 'Borough', 'Neighborhood', 'Latitude', 'Longitude'

B. For population data we use kaggle's borough-wise population in new york. 
https://www.kaggle.com/new-york-city/new-york-city-population

C For data of Toronto we use the following link.
"https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

D. For the geographical coordinates of the neighborhoods we use the following link “https://cocl.us/Geospatial_data”.

E. For population in Scarborough including their ehnicity we use the link-
  “Demographics of Toronto” (https://en.m.wikipedia.org/wiki/Demographics_of_Toronto#Ethnic_diversity) 
  
F. For getting location and other information about various venues in Toronto Foursquare’s explore API is used. Using the Foursquare’s explore API we got details about the venues up present in Toronto and collected their names, categories and locations (latitude and longitude).Foursquare API link is https://developer.foursquare.com/docs.

## Importing libraries before downloading and exploring 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Collection of Manhatton data
1. Download and Explore Dataset

New York has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

In [2]:
import wget

In [3]:
path_nyc = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json"

In [4]:
nyc_json_file = wget.download(path_nyc)

100% [............................................................................] 115774 / 115774

In [5]:
nyc_json_file

'newyork_data (2).json'

In [6]:
# let us load the data
with open('newyork_data.json') as json_data:
    nyc_data = json.load(json_data)
    

In [7]:
nyc_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [8]:
nyc_neigh = nyc_data['features']

#### Transform the data into a pandas dataframe

In [9]:
# create a empty dataframe with columns Borough, Neighborhoods, Lattitude and Longitude
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [10]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Now let's loop through the data and fill the dataframe one row at a time.

In [11]:
nyc_neigh[0]['properties']

{'name': 'Wakefield',
 'stacked': 1,
 'annoline1': 'Wakefield',
 'annoline2': None,
 'annoline3': None,
 'annoangle': 0.0,
 'borough': 'Bronx',
 'bbox': [-73.84720052054902,
  40.89470517661,
  -73.84720052054902,
  40.89470517661]}

In [12]:
for data in nyc_neigh:
    borough =  data['properties']['borough']
    neighborhood = data['properties']['name']
    
    lat_long = data['geometry']['coordinates']
    lattitude = lat_long[1]
    longitude = lat_long[0]
    
    neighborhoods = neighborhoods.append({'Borough':borough,
                          'Neighborhood':neighborhood, 
                          'Latitude':lattitude, 'Longitude':longitude}, ignore_index = True)


In [13]:
neighborhoods.shape

(306, 4)

In [14]:
#df.drop(df.index, inplace=True)
#f.drop(df.iloc[0:0], inplace=True)
#neighborhoods.drop(neighborhoods.index, inplace = True)

#neighborhoods


In [15]:
## get manhatton data
manhattan_data = neighborhoods[neighborhoods['Borough']== 'Manhattan']
manhattan_data.reset_index(drop= True, inplace =True)

In [16]:
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [17]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [18]:
LIMIT = 100
radius = 500
CLIENT_ID = '0XGO3EFCURDXQET2DLDCN2PX2QOLOGXJ3V2CKYGE1FYOIYD5' # your Foursquare ID
CLIENT_SECRET = 'Y2JZJUJRQQQQWDSQB4FJCDWZAXIMAHTD5PFDOZIEYLN2OBEK' # your Foursquare Secret
ACCESS_TOKEN = '4RCIFXGTD1FAJEXTO3JZPCCG3S5I2STLFW5BIKIHQIK5N2OG' # your FourSquare Access Token
VERSION = '20180605'
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=0XGO3EFCURDXQET2DLDCN2PX2QOLOGXJ3V2CKYGE1FYOIYD5&client_secret=Y2JZJUJRQQQQWDSQB4FJCDWZAXIMAHTD5PFDOZIEYLN2OBEK&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6087cb55c7898674fe7b8747'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 22,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4baf59e8f964a520a6f93be3',
       'name': 'Bikram Yoga',
       'location': {'address': '5500 Broadway',
        'crossStreet': '230th Street',
        'lat': 40.876843690797934,
        'lng': -73.90620384419528,
        'labeledLatLngs': [{'la

#### From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Bikram Yoga,Yoga Studio,40.876844,-73.906204
1,Arturo's,Pizza Place,40.874412,-73.910271
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Dunkin',Donut Shop,40.877136,-73.906666
4,Astral Fitness & Wellness Center,Gym,40.876705,-73.906372


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [24]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
1,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Astral Fitness & Wellness Center,40.876705,-73.906372,Gym


In [25]:
print(manhattan_venues.shape)

(3229, 7)


## How many restaurants are in Manhattan ?

In [26]:
restaurant_man = manhattan_venues[manhattan_venues['Venue Category'].str.contains('Restaurant')]

In [27]:
no_of_restaurant =restaurant_man['Venue Category'].count()
print("There are {} restaurant in Manhatton.".format(no_of_restaurant))

There are 886 restaurant in Manhatton.


## Let's see how many people live in Manhattan

In [28]:
# get the data from url https://www2.census.gov/library/publications/2001/compendia/ccdb00/tabA2.pdf

# new-york-city-population-by-borough-1950-2040.csv  give the link github

In [29]:
df_man_pop = pd.read_csv(r"C:/Users/User/Documents/1.Mukta IBM/Capstone Projecct/week4-5/data/nyc_pop_by_borough/new-york-city-population-by-borough-1950-2040.csv")

In [30]:
df_man_pop.head(3)

,Age Group,Borough,1950,1950 - Boro share of NYC total,1960,1960 - Boro share of NYC total,1970,1970 - Boro share of NYC total,1980,1980 - Boro share of NYC total,1990,1990 - Boro share of NYC total,2000,2000 - Boro share of NYC total,2010,2010 - Boro share of NYC total,2020,2020 - Boro share of NYC total,2030,2030 - Boro share of NYC total,2040,2040 - Boro share of NYC total
0,Total Population,NYC Total,7891957,100.00,7781984,100.00,7894862,100.00,7071639,100.00,7322564,100.00,8008278,100.00,8242624,100.00,8550971,100.00,8821027,100.00,9025145,100.00
1,Total Population,Bronx,1451277,18.39,1424815,18.31,1471701,18.64,1168972,16.53,1203789,16.44,1332650,16.64,1385108,16.80,1446788,16.92,1518998,17.22,1579245,17.50
2,Total Population,Brooklyn,2738175,34.70,2627319,33.76,2602012,32.96,2230936,31.55,2300664,31.42,2465326,30.78,2552911,30.97,2648452,30.97,2754009,31.22,2840525,31.47


In [31]:
nyc_pop_2020 =df_man_pop[['Borough','2020']]
nyc_pop_2020

,Borough,2020
0,NYC Total,8550971
1,Bronx,1446788
2,Brooklyn,2648452
3,Manhattan,1638281
4,Queens,2330295
5,Staten Island,487155


In [32]:
nyc_pop_2020['Borough'][3]

'   Manhattan'

In [33]:
nyc_pop_2020['Borough'] = nyc_pop_2020['Borough'].str.strip()

C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
#print("There are {} people live in Manhattan.".format(poulation_in_mahattan))

In [35]:
man_hattan_pop =nyc_pop_2020[nyc_pop_2020['Borough']=='Manhattan']['2020']

In [36]:
man_hattan_pop = man_hattan_pop.values[0] 

In [97]:
ratio_of_restaurant_to_pop_man = no_of_restaurant/man_hattan_pop
ratio_of_restaurant_to_pop_man

0.000540810764453717

In [38]:
print("The population size of Manhattan is {}. ".format(man_hattan_pop))

The population size of Manhattan is 1638281. 


In [98]:
print("The ratio of number of restaurant to population size in Manhattan is {:.5f}.".format(ratio_of_restaurant_to_pop_man))

The ratio of number of restaurant to population size in Manhattan is 0.00054.


## Collection of Scarborough data
1. Download and Explore Dataset

Toronto has a total of 15 boroughs in Toronto City. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 15 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

In [40]:
 url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [41]:
table_main = pd.read_html(url)

In [42]:
len(table_main)

3

In [43]:
table_main[0][0:3]

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...


In [44]:
table_main[1]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,NaN,Canadian postal codes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...,NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...,NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NL,NS,PE,NB,QC,QC,QC,ON,ON,ON,ON,ON,MB,SK,AB,BC,NU/NT,YT
3,A,B,C,E,G,H,J,K,L,M,N,P,R,S,T,V,X,Y


In [45]:
table_main[2]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,NL,NS,PE,NB,QC,QC,QC,ON,ON,ON,ON,ON,MB,SK,AB,BC,NU/NT,YT
1,A,B,C,E,G,H,J,K,L,M,N,P,R,S,T,V,X,Y


### we are interested in table_main[0]

In [46]:
new_table= table_main[0]
new_table.head()

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned


In [47]:
no_of_col = new_table.shape[1]
no_of_col

9

In [48]:
# each column has postal code, borough name and neighborhood. so data of all columns to be taken in one column
# create an empty series.
s= pd.Series()
# appending this series with all columns from 0 to 9
for i in range(no_of_col):
    s = s.append(new_table[i])
# creating dataframe from this series
df_toronto = pd.DataFrame(s)
df_toronto.reset_index(drop=True, inplace = True) 
df_toronto.tail()

C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


,0
175,M9VEtobicoke(South Steeles / Silverstone / Hum...
176,M9WEtobicokeNorthwest(Clairville / Humberwood ...
177,M9XNot assigned
178,M9YNot assigned
179,M9ZNot assigned


In [49]:
# it is nticed that first 3 character is postal code, then bfore'(' is borough name, rest is neighbourhood
df_toronto = df_toronto.assign(PostalCode = df_toronto[0].str[0:3], rest = df_toronto[0].str[3:])
df_toronto.head()


,0,PostalCode,rest
0,M1ANot assigned,M1A,Not assigned
1,M1BScarborough(Malvern / Rouge),M1B,Scarborough(Malvern / Rouge)
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M1C,Scarborough(Rouge Hill / Port Union / Highland...
3,M1EScarborough(Guildwood / Morningside / West ...,M1E,Scarborough(Guildwood / Morningside / West Hill)
4,M1GScarborough(Woburn),M1G,Scarborough(Woburn)


In [50]:
# spliting in the first apppearane of "("
df_toronto[['Borough', 'Neighborhood']] = df_toronto.rest.apply(lambda x: pd.Series(str(x).split("(", 1)))

In [51]:
df_toronto.head()

,0,PostalCode,rest,Borough,Neighborhood
0,M1ANot assigned,M1A,Not assigned,Not assigned,NaN
1,M1BScarborough(Malvern / Rouge),M1B,Scarborough(Malvern / Rouge),Scarborough,Malvern / Rouge)
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M1C,Scarborough(Rouge Hill / Port Union / Highland...,Scarborough,Rouge Hill / Port Union / Highland Creek)
3,M1EScarborough(Guildwood / Morningside / West ...,M1E,Scarborough(Guildwood / Morningside / West Hill),Scarborough,Guildwood / Morningside / West Hill)
4,M1GScarborough(Woburn),M1G,Scarborough(Woburn),Scarborough,Woburn)


In [52]:
df_toronto.drop(columns= [0,'rest'], inplace = True)

In [53]:
df_toronto['Neighborhood'] = df_toronto['Neighborhood'].str.replace(")", "",)

In [54]:
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M1B,Scarborough,Malvern / Rouge
2,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
3,M1E,Scarborough,Guildwood / Morningside / West Hill
4,M1G,Scarborough,Woburn


In [55]:
# removing Borough with 'Not Assigned'
df_toronto = df_toronto[df_toronto['Borough']!='Not assigned']
df_toronto.head()

,PostalCode,Borough,Neighborhood
1,M1B,Scarborough,Malvern / Rouge
2,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
3,M1E,Scarborough,Guildwood / Morningside / West Hill
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [56]:
df_toronto.reset_index(drop = True, inplace = True)
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Now we have cleaned neighborhood data of Toronto

In [57]:
df_toronto.shape

(103, 3)

### Reading the Geospatial_Coordinates .csv for incorporating in the main dataframe  this 

In [58]:
#This csv file is downloaded from ibm link

In [59]:
path_coordinates = "D:/dataset_selected/Geospatial_Coordinates .csv"

In [60]:
location = pd.read_csv(path_coordinates)
location.rename(columns ={'Postal Code': 'PostalCode'}, inplace = True)
location.head(3)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711


### Merging two dataframe on column 'PostalCode'

In [61]:
df_toronto_location = pd.merge(df_toronto, location)
df_toronto_location.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [62]:
neighborhood_lat = df_toronto_location.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_long = df_toronto_location.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto_location.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_lat, 
                                                               neighborhood_long))

Latitude and longitude values of Malvern / Rouge are 43.806686299999996, -79.19435340000001.


In [63]:
LIMIT = 100
radius = 500
CLIENT_ID = '0XGO3EFCURDXQET2DLDCN2PX2QOLOGXJ3V2CKYGE1FYOIYD5' # your Foursquare ID
CLIENT_SECRET = 'Y2JZJUJRQQQQWDSQB4FJCDWZAXIMAHTD5PFDOZIEYLN2OBEK' # your Foursquare Secret
ACCESS_TOKEN = '4RCIFXGTD1FAJEXTO3JZPCCG3S5I2STLFW5BIKIHQIK5N2OG' # your FourSquare Access Token
VERSION = '20180605'
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat, 
    neighborhood_long, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=0XGO3EFCURDXQET2DLDCN2PX2QOLOGXJ3V2CKYGE1FYOIYD5&client_secret=Y2JZJUJRQQQQWDSQB4FJCDWZAXIMAHTD5PFDOZIEYLN2OBEK&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [74]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6087ceb7bd1da232f0f6fbb1'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [75]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


In [69]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [70]:
scarborough_venues = getNearbyVenues(names=df_toronto_location['Neighborhood'],
                                   latitudes=df_toronto_location['Latitude'],
                                   longitudes=df_toronto_location['Longitude']
                                  )

Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview / Henry Farm / Oriole
Bayview Village
York Mills / Silver Hills
Willowdale / Newtonbrook
WillowdaleSouth
York Mills West
WillowdaleWest
Parkwoods
Don MillsNorth
Don MillsSouth(Flemingdon Park
Bathurst Manor / Wilson Heights / Downsview North
Northwood Park / York University
DownsviewEast (CFB Toronto
DownsviewWest
DownsviewCentral
DownsviewNorthwest
Victoria Village
Parkview Hill / Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Par

In [76]:
scarborough_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malvern / Rouge,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Guildwood / Morningside / West Hill,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,Guildwood / Morningside / West Hill,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,Guildwood / Morningside / West Hill,43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


In [77]:
print(scarborough_venues.shape)

(2110, 7)


In [78]:
restaurant_scarborough = scarborough_venues[scarborough_venues['Venue Category'].str.contains('Restaurant')]

In [107]:
no_of_restaurant =restaurant_scarborough['Venue Category'].count()
print("There are {} restaurant in Scarborough.".format(no_of_restaurant))

There are 472 restaurant in Scarborough.


### Let's see how many people live in Scarborough

In [81]:
### scrapping 
import wikipedia as wp

In [82]:
html = wp.page("Demographics of Toronto").html().encode("UTF-8")

In [84]:
len(html)

315399

In [86]:
Scar_population_df = pd.read_html(html, header = 0)[15]

In [90]:
Scar_population_df

,Riding,Population,Ethnic Origin #1,%,Ethnic Origin #2,%.1,Ethnic Origin #3,%.2,Ethnic Origin #4,%.3,Ethnic Origin #5,%.4,Ethnic Origin #6,%.5,Ethnic Origin #7,%.6,Ethnic Origin #8,%.7
0,Etobicoke-Lakeshore,127520,English,17.1,Canadian,15.9,Irish,14.4,Scottish,13.5,Polish,9.2,Italian,9.1,Ukrainian,7.6,German,7.1
1,Etobicoke North,116960,East Indian,22.2,Canadian,7.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Etobicoke Centre,116055,Italian,15.1,English,14.3,Canadian,12.1,Irish,10.8,Scottish,10.4,Ukrainian,8.1,Polish,7.4,NaN,NaN
3,York South-Weston,115130,Portuguese,14.5,Italian,12.8,Canadian,8.7,Jamaican,8.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
Scarborough_ttl_pop = Scar_population_df['Population'].sum()

In [93]:
print("Total Population in Scarborough is {}".format(Scarborough_ttl_pop))

Total Population in Scarborough is 475665


In [109]:
ratio_of_scar_restaurant_to_pop = no_of_restaurant/Scarborough_ttl_pop

In [110]:
ratio_of_scar_restaurant_to_pop

0.0009922949975297742

In [111]:
print("The ratio of restaurant to population in Scarborough is {:.5f} ".format(ratio_of_scar_restaurant_to_pop))

The ratio of restaurant to population in Scarborough is 0.00099 


In [99]:
print("The ratio of number of restaurant to population size in Manhattan is {:.5f}.".format(ratio_of_restaurant_to_pop_man))

The ratio of number of restaurant to population size in Manhattan is 0.00054.


In [113]:
times_heigher = ratio_of_scar_restaurant_to_pop/ratio_of_restaurant_to_pop_man
times_heigher

1.8348284885418467

In [115]:
print("The ratio of restaurant to population size in Scarborough, Toronto is {:.3f} higher than that of Manhattan, New York". format(times_heigher))

The ratio of restaurant to population size in Scarborough, Toronto is 1.835 higher than that of Manhattan, New York


## Report on the analysis
### result and discussion
Our objective was to select the borough where a restaurant would be started. We initially took took two boroughs; Manhattan
in New York and Scarborough in Toronto. Population size and number of existing restaurant was considered the major factor. 
For population size we used published data sets from kaggle, “Demographics of Toronto”. Number of restaurant including its
location was retrieved using Forsquare API.
The dataset used for Scarborough published in 2016. This is a gap for this analysis. We assumed that all other factor will remain 
same. We found that there are 472 restaurant for 475665 people in Scarborough whereas the number of restaurant in Manhattan is
886 for 1638281 people. 
The ratio of number of restaurant to population size in Manhattan is 0.00054 while it is 0.00099 for Scarborough.
The ratio of restaurant to population in Scarborough is 1.835 times higher than that of Manhattan

#### Conlusion:
From the above discussion it is conluded that starting resaurant in Manhattan is better than starting at Scarborough.
